<a href="https://colab.research.google.com/github/v41/johan/blob/main/35_tiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math, json
import numpy as np
from shapely import geometry

In [ ]:
earthRadius = 6378137
quadrant = 2 * math.pi * earthRadius / 2

def lonToX(lon):
  'Converts given lon in WGS84 Datum to X in Spherical Mercator EPSG:3857'
  x = lon * quadrant / 180
  return x

def latToY(lat):
  'Converts given lat in WGS84 Datum to Y in Spherical Mercator EPSG:3857'
  y = math.log( math.tan((90 + lat) * math.pi / 360 )) / (math.pi / 180)
  y = y * quadrant / 180
  return y

def xToLon(x):
  'Converts X from Spherical Mercator EPSG:3857 to lon in WGS84 Datum'
  lon = x / quadrant * 180
  return lon

def yToLat(y):
  'Converts Y from Spherical Mercator EPSG:3857 to lat in WGS84 Datum'
  lat = y / quadrant * 180
  lat = 180 / math.pi * (2 * math.atan( math.exp(lat * math.pi / 180)) - math.pi / 2)
  return lat

def genTiles(xMin, yMin, xMax, yMax, n=3):
  'Generate tiles based on number of n'
  xDelta = (xMax - xMin) / n
  yDelta = (yMax - yMin) / n
  xRange = np.arange(xMin, xMax, xDelta) #min to max
  yRange = np.arange(yMax, yMin, -yDelta) #max to min
    return [(i, j-yDelta, i+xDelta, j) for j in yRange[:n] for i in xRange[:n]]

def getTile(n=3):
  'Tile index mapping based on n*n'
  if n == 2:
    t = list('1234')
  elif n == 3:
    t = list('123456789')
  elif n == 4:
    t = list('0123456789abcdef')
  elif n == 5:
    t = list('abcdefghijklmnopqrstuvwxy')
  elif n == 6:
    t = list('0123456789abcdefghijklmnopqrstuvwxyz')
  elif n == 7: #unstable!
    t = list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVW')
  elif n == 8:
    t = list('0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ?@')
  return t

def findIndex(longitude,latitude,zoom, geom=False,full=False):
  x, y = lonToX(longitude), latToY(latitude)  
  i = 1
  tiles = ''
  nf = 5
  nb = 3
  gtf = getTile(nf)
  gtb = getTile(nb)
  polygons = []
  xMin, yMin, xMax, yMax = -quadrant, -quadrant, quadrant, quadrant
  while i <= zoom:
    if i <= 4:
      for t, bound in zip(gtf, genTiles(xMin, yMin, xMax, yMax, n=nf)):
        if full:
          polygon = geometry.mapping(geometry.box(xToLon(bound[0]), yToLat(bound[1]), xToLon(bound[2]), yToLat(bound[3])))      
          polygons.append(dict(geometry=polygon, id=tiles+t, type='Feature'))
        if bound[0] < x < bound[2] and bound[1] < y < bound[3]:
          xMin, yMin, xMax, yMax = bound #tile = '{0:x}'.format(t) for n=4 with hex, start from 0
          tile = t
#          break
    elif i > 4:
      for t, bound in zip(gtb, genTiles(xMin, yMin, xMax, yMax, n=nb)):
        if full:
          polygon = geometry.mapping(geometry.box(xToLon(bound[0]), yToLat(bound[1]), xToLon(bound[2]), yToLat(bound[3])))      
          polygons.append(dict(geometry=polygon, id=tiles+t, type='Feature'))
        if bound[0] < x < bound[2] and bound[1] < y < bound[3]:
          xMin, yMin, xMax, yMax = bound #tile = '{0:x}'.format(t) for n=4 with hex, start from 0
          tile = t
#          break
    i += 1
    tiles += tile # increment tile prefix

    if geom:
      polygon = geometry.mapping(geometry.box(xToLon(xMin), yToLat(yMin), xToLon(xMax), yToLat(yMax)))
      polygons.append(dict(type='Feature', geometry=polygon, id=tiles))

  if geom:
    geomCol = dict(type='FeatureCollection', features=polygons)
    with open(f"{tiles}_min.geojson", "w") as filename:
      json.dump(geomCol, filename)

  if full:
    geomCol = dict(type='FeatureCollection', features=polygons)
    with open(f"{tiles}_full.geojson", "w") as filename:
      json.dump(geomCol, filename)

#  f = 4
  return tiles #'-'.join(tiles[i:i+f] for i in range(0,len(tiles),f))

In [ ]:
findIndex(106.9,-6.5,12)